# 목차

# 1. 데이터 불러오기
 - column 이름 뽑을 df 부르기
 - 새로운 이름으로 파일 저장하기
 
# 2. 이제 본격적으로 dataframe을 만들어봅시다
 - 필요한 column만 빼주기
 
# 3. 이제 FE를 해봅시다
 - mile data
   - mile에 course_code 완성해주기
   - hakno, bb, sbb 합쳐서 course code 만들어주기
   - mile df categorical 변수 0, 1로 변환하기
   - 학년 합쳐주기 (4+ == 4, 1<= == 1)
   - mile df 학년 column dummy 변수화 시키기	
 - meta data
   - meta df 복수전공 여부에 따라서 0, 1로 바꿔주기
 - Data 저장하기
 
# 4. Merge 시켜주기 
 - meta df 중복값 없애고서 merge
 - 학년별 정원 제한 없는 과목은 NaN값으로 바꿔주기
 - merge 해주기
 - meta data가 없는 row들 없애주기
 - Data 저장하기
 
# 5. 데이터 확인


In [1]:
# 기본
import numpy as np
from numpy.linalg import inv
from numpy.linalg import solve
import random

import pandas as pd
from pandas import Series, DataFrame
import pandas_profiling
import openpyxl

import matplotlib.pyplot as plt
import urllib
import matplotlib.dates as mdates
from matplotlib.dates import bytespdate2num
import matplotlib.ticker as ticker
from matplotlib import font_manager, rc
from matplotlib import style
import seaborn as sns

# import itertools
# import re
# import warnings
# warnings.filterwarnings("ignore")

# # 시계열 데이터 처리
# import calendar
# import dateutil
# from dateutil.parser import parse
# import datetime

# # Network 분석
# import networkx as nx

# # 지도시각화
# import folium
# from folium import plugins
# import html
# import json
# import geopy
# from geopy.geocoders import Nominatim
# import os
# import requests
# import ipywidgets
# from IPython.display import Image
# from ipywidgets import interact

# # Clustering
# from sklearn.cluster import KMeans
# from scipy.cluster.hierarchy import linkage, dendrogram

# # Factor analysis
# from factor_analyzer import FactorAnalyzer

# Machine learning 용도
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.model_selection import PredefinedSplit
from sklearn.model_selection import GridSearchCV, ParameterGrid
from sklearn.metrics import (roc_curve, auc, accuracy_score, roc_auc_score)
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.datasets import make_blobs
import scipy.cluster.hierarchy as shc
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import classification_report
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.ensemble import AdaBoostRegressor, GradientBoostingRegressor
from lightgbm import LGBMClassifier, LGBMRegressor
import lightgbm as lgb

# # Deep learning 용도
# import torch
# import torchvision.datasets as dsets
# import torchvision.transforms as transforms
# import random
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# import torch.optim as optim

# # Wordcloud
# import konlpy
# from konlpy.tag import Okt
# import collections
# from collections import Counter
# from wordcloud import WordCloud, STOPWORDS as stopwords
# from PIL import Image, ImageFilter
# from wordcloud import ImageColorGenerator
# import pickle

# # 한글 사용하기
# import matplotlib.font_manager as fm
# from matplotlib import rc
# font_name = fm.FontProperties(fname="c:/Windows/Fonts/malgun.ttf").get_name()
# rc('font', family=font_name)

# # 크롤링
#  from selenium import webdriver
# import requests
# from bs4 import BeautifulSoup

# # 기타
# from urllib.request import urlopen
from tqdm import tqdm
# import time
# from zeep import Client
# from collections import namedtuple
# import sqlite3

# # 한 번에 matplotlib 그림 띄우기
# %matplotlib inline    
# %config InlineBackend.figure_format = 'retina'  #%matplotlib 뒤에 써주면 그래프를 더 높은 해상도로 보여줌

# 설정 관련
pd.set_option('display.max_columns', 200)
pd.set_option('display.max_rows', 200)

# 1. 데이터 불러오기

In [459]:
meta_original = pd.read_csv('meta_table.csv', header=None)
mile_original = pd.read_csv('mile_table.csv', header=None)   

확인해보니 column 이름이 없었어요! 그래서 만들어줬지롱~~

## column 이름 뽑을 df 부르기

In [460]:
meta_column = pd.read_csv('20161meta.csv', header=None)
mile_column = pd.read_csv('20161mile.csv', header=None)   

columns 이름들 dict으로 만들어주기

In [461]:
new_name_meta = dict(zip(list(meta_original.columns), list(meta_column.columns)))
new_name_mile = dict(zip(list(mile_original.columns), list(mile_column.columns)))

In [462]:
meta = meta.rename(columns = new_name_meta)
mile = mile.rename(columns = new_name_mile)

In [463]:
meta.head(3)

,hyhg,course_code,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,20161,OMA1001-01-00,"김종철,홍기원",63,40,0,0,0,21,21,21,36,12.70
1,20161,OMA1002-01-00,우환식,63,98,0,0,0,21,21,21,36,20.10
2,20161,YCA1101-10-00,조재국,53,43,0,0,0,18,17,18,36,12.98


In [464]:
mile.head(3)

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,course_code,grade_1,grade_2,grade_3,grade_4
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,OMA1001-01-00,0,0,0,1
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,OMA1001-01-00,0,0,0,1
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,OMA1001-01-00,0,0,0,1


## 새로운 이름으로 파일 저장하기

In [465]:
meta.to_csv('meta_table_new_column.csv', index=False)
mile.to_csv('mile_table_new_column.csv', index=False)

# 2. 이제 본격적으로 dataframe을 만들어봅시다

data 불러주기

In [6]:
meta_original = pd.read_csv('meta_table_new_column.csv')
mile_original = pd.read_csv('mile_table_new_column.csv')   

In [7]:
meta

,hyhg,course_code,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,20161,OMA1001-01-00,"김종철,홍기원",63,40,0,0,0,21,21,21,36,12.70
1,20161,OMA1002-01-00,우환식,63,98,0,0,0,21,21,21,36,20.10
2,20161,YCA1101-10-00,조재국,53,43,0,0,0,18,17,18,36,12.98
3,20161,YCA1101-11-00,박정세,24,48,0,0,0,8,8,8,36,16.56
4,20161,YCA1101-12-00,민경식,50,14,50,0,0,0,0,0,36,7.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26581,20192,KLI1002-01-00,김성숙,80,45,0,0,0,0,0,0,36,5.47
26582,20192,KLI1003-01-00,김성숙,50,32,0,0,0,0,0,0,36,3.34
26583,20192,KLI1004-01-00,김성숙,40,24,0,0,0,0,0,0,36,4.08
26584,20192,KLI1005-01-00,김성숙,15,9,0,0,0,0,0,0,36,1.67


In [468]:
mile

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,course_code,grade_1,grade_2,grade_3,grade_4
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,OMA1001-01-00,0,0,0,1
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,OMA1001-01-00,0,0,0,1
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,OMA1001-01-00,0,0,0,1
3,O,20161,36,0,0,5,0,1,0.7642,0.8421,OMA1001-01-00,0,0,0,1
4,O,20161,36,0,0,5,0,1,0.4337,0.4210,OMA1001-01-00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,1,0,0,5,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1
985317,O,20192,1,0,0,5,0,1,0.2500,0.9375,KLI1006-01-00,0,1,0,0
985318,O,20192,1,0,0,4,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1
985319,O,20192,1,0,0,4,0,1,0.7500,0.8750,KLI1006-01-00,0,0,0,1


## 필요한 column만 빼주기

meta 사용 columns

	학정번호	
	과목명	
	담당교수	
	정원, 참여인원 -> 1개로 합칠지 말지 고민
	전공자 
	정원	
	학년별 정원	
	Max 마일리지	
	average

In [472]:
meta = meta[['hyhg', 'course_code', 'instructor', 'quota', 'participants', 
             'major_quota', 'second_major','grade_quota_1', 'grade_quota_2',
             'grade_quota_3', 'grade_quota_4','max_mileage', 'average']]

In [473]:
mile

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,course_code,grade_1,grade_2,grade_3,grade_4
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,OMA1001-01-00,0,0,0,1
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,OMA1001-01-00,0,0,0,1
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,OMA1001-01-00,0,0,0,1
3,O,20161,36,0,0,5,0,1,0.7642,0.8421,OMA1001-01-00,0,0,0,1
4,O,20161,36,0,0,5,0,1,0.4337,0.4210,OMA1001-01-00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,1,0,0,5,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1
985317,O,20192,1,0,0,5,0,1,0.2500,0.9375,KLI1006-01-00,0,1,0,0
985318,O,20192,1,0,0,4,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1
985319,O,20192,1,0,0,4,0,1,0.7500,0.8750,KLI1006-01-00,0,0,0,1


mile 사용 columns

	수강성공여부	
	hyhg(연도.학기)	
	학정번호
    bb
    sbb
	마일리지	
	전공여부
	복수전공여부	
	신청과목수	
	졸업신청		
	수강여부
	credits_rate	
	previous_credits_rate	
	학년

In [ ]:
mile = mile[['enrolled', 'hyhg', 'hakno', 'bb', 'sbb', 'mileage', 'major',
       'double', 'enrolled_courses', 'graduation', 'first_enroll',
       'credits_rate', 'previous_credits_rate', 'grade']]

# 3. 이제 FE를 해봅시다

## mile data

In [ ]:
mile.head()

### mile에 course_code 완성해주기

In [ ]:
temp = mile.copy()

In [ ]:
temp.dtypes

In [ ]:
temp.loc[temp['bb']<10, 'bb'] = '0' + temp['bb'].astype(str)

In [ ]:
temp.loc[temp['sbb']<10, 'sbb'] = '0' + temp['sbb'].astype(str)

In [288]:
temp

,enrolled,hyhg,hakno,bb,sbb,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade
0,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7925,0.9473,4
1,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7857,0.7142,4
2,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7428,0.8947,4
3,O,20161,OMA1001,01,00,36,N,N,5,N,Y,0.7642,0.8421,4
4,O,20161,OMA1001,01,00,36,N,N,5,N,Y,0.4337,0.4210,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,KLI1006,01,00,1,N,N,5,N,Y,0.7500,0.9375,4
985317,O,20192,KLI1006,01,00,1,N,N,5,N,Y,0.2500,0.9375,2
985318,O,20192,KLI1006,01,00,1,N,N,4,N,Y,0.7500,0.9375,4
985319,O,20192,KLI1006,01,00,1,N,N,4,N,Y,0.7500,0.8750,4


In [292]:
temp.dtypes

enrolled                  object
hyhg                       int64
hakno                     object
bb                        object
sbb                       object
mileage                    int64
major                     object
double                    object
enrolled_courses           int64
graduation                object
first_enroll              object
credits_rate             float64
previous_credits_rate    float64
grade                      int64
dtype: object

In [294]:
temp

,enrolled,hyhg,hakno,bb,sbb,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade
0,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7925,0.9473,4
1,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7857,0.7142,4
2,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7428,0.8947,4
3,O,20161,OMA1001,01,00,36,N,N,5,N,Y,0.7642,0.8421,4
4,O,20161,OMA1001,01,00,36,N,N,5,N,Y,0.4337,0.4210,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,KLI1006,01,00,1,N,N,5,N,Y,0.7500,0.9375,4
985317,O,20192,KLI1006,01,00,1,N,N,5,N,Y,0.2500,0.9375,2
985318,O,20192,KLI1006,01,00,1,N,N,4,N,Y,0.7500,0.9375,4
985319,O,20192,KLI1006,01,00,1,N,N,4,N,Y,0.7500,0.8750,4


### hakno, bb, sbb 합쳐서 course code 만들어주기

In [302]:
temp_2 = temp.copy()

In [303]:
temp_2.dtypes

enrolled                  object
hyhg                       int64
hakno                     object
bb                        object
sbb                       object
mileage                    int64
major                     object
double                    object
enrolled_courses           int64
graduation                object
first_enroll              object
credits_rate             float64
previous_credits_rate    float64
grade                      int64
dtype: object

In [305]:
temp_2['hakno'] = temp_2['hakno'].astype(str)
temp_2['bb'] = temp_2['bb'].astype(str)
temp_2['sbb'] = temp_2['sbb'].astype(str)

In [306]:
temp_2['course_code'] = temp_2['hakno'] + '-' + temp_2['bb'] + '-' + temp_2['sbb']

In [308]:
temp_2

,enrolled,hyhg,hakno,bb,sbb,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade,course_code
0,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7925,0.9473,4,OMA1001-01-00
1,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7857,0.7142,4,OMA1001-01-00
2,O,20161,OMA1001,01,00,36,N,N,6,N,Y,0.7428,0.8947,4,OMA1001-01-00
3,O,20161,OMA1001,01,00,36,N,N,5,N,Y,0.7642,0.8421,4,OMA1001-01-00
4,O,20161,OMA1001,01,00,36,N,N,5,N,Y,0.4337,0.4210,3,OMA1001-01-00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,KLI1006,01,00,1,N,N,5,N,Y,0.7500,0.9375,4,KLI1006-01-00
985317,O,20192,KLI1006,01,00,1,N,N,5,N,Y,0.2500,0.9375,2,KLI1006-01-00
985318,O,20192,KLI1006,01,00,1,N,N,4,N,Y,0.7500,0.9375,4,KLI1006-01-00
985319,O,20192,KLI1006,01,00,1,N,N,4,N,Y,0.7500,0.8750,4,KLI1006-01-00


In [310]:
mile = temp_2.copy()

In [312]:
del mile['hakno']
del mile['bb']
del mile['sbb']

In [315]:
meta

,hyhg,course_code,instructor,quota,participants,major_quota,second_major,grande_1,grade_2,grade_3,grade_4,max_mileage,average
0,20161,OMA1001-01-00,"김종철,홍기원",63,40,0,N,0,21,21,21,36,12.70
1,20161,OMA1002-01-00,우환식,63,98,0,N,0,21,21,21,36,20.10
2,20161,YCA1101-10-00,조재국,53,43,0,N,0,18,17,18,36,12.98
3,20161,YCA1101-11-00,박정세,24,48,0,N,0,8,8,8,36,16.56
4,20161,YCA1101-12-00,민경식,50,14,50,N,0,0,0,0,36,7.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26581,20192,KLI1002-01-00,김성숙,80,45,0,N,0,0,0,0,36,5.47
26582,20192,KLI1003-01-00,김성숙,50,32,0,N,0,0,0,0,36,3.34
26583,20192,KLI1004-01-00,김성숙,40,24,0,N,0,0,0,0,36,4.08
26584,20192,KLI1005-01-00,김성숙,15,9,0,N,0,0,0,0,36,1.67


In [313]:
mile

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade,course_code
0,O,20161,36,N,N,6,N,Y,0.7925,0.9473,4,OMA1001-01-00
1,O,20161,36,N,N,6,N,Y,0.7857,0.7142,4,OMA1001-01-00
2,O,20161,36,N,N,6,N,Y,0.7428,0.8947,4,OMA1001-01-00
3,O,20161,36,N,N,5,N,Y,0.7642,0.8421,4,OMA1001-01-00
4,O,20161,36,N,N,5,N,Y,0.4337,0.4210,3,OMA1001-01-00
...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,1,N,N,5,N,Y,0.7500,0.9375,4,KLI1006-01-00
985317,O,20192,1,N,N,5,N,Y,0.2500,0.9375,2,KLI1006-01-00
985318,O,20192,1,N,N,4,N,Y,0.7500,0.9375,4,KLI1006-01-00
985319,O,20192,1,N,N,4,N,Y,0.7500,0.8750,4,KLI1006-01-00


### mile df categorical 변수 0, 1로 변환하기

In [318]:
mile.head(3)

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade,course_code
0,O,20161,36,N,N,6,N,Y,0.7925,0.9473,4,OMA1001-01-00
1,O,20161,36,N,N,6,N,Y,0.7857,0.7142,4,OMA1001-01-00
2,O,20161,36,N,N,6,N,Y,0.7428,0.8947,4,OMA1001-01-00


In [319]:
dict_ny = {'N':0, "Y":1}

In [320]:
mile['double'] = mile['double'].map(dict_ny)
mile['major'] = mile['major'].map(dict_ny)
mile['graduation'] = mile['graduation'].map(dict_ny)
mile['first_enroll'] = mile['first_enroll'].map(dict_ny)

In [321]:
mile.head(3)

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade,course_code
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,4,OMA1001-01-00
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,4,OMA1001-01-00
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,4,OMA1001-01-00


### 학년 합쳐주기 (4+ == 4, 1<= == 1)

In [324]:
mile['grade'].value_counts()

4    349707
2    342488
3    286770
5      4337
6      1782
1       159
0        78
Name: grade, dtype: int64

In [325]:
dict_grade = {0:1, 1:1, 2:2, 3:3, 4:4, 5:4, 6:4}

In [326]:
mile['grade'] = mile['grade'].map(dict_grade)

In [327]:
mile

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade,course_code
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,4,OMA1001-01-00
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,4,OMA1001-01-00
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,4,OMA1001-01-00
3,O,20161,36,0,0,5,0,1,0.7642,0.8421,4,OMA1001-01-00
4,O,20161,36,0,0,5,0,1,0.4337,0.4210,3,OMA1001-01-00
...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,1,0,0,5,0,1,0.7500,0.9375,4,KLI1006-01-00
985317,O,20192,1,0,0,5,0,1,0.2500,0.9375,2,KLI1006-01-00
985318,O,20192,1,0,0,4,0,1,0.7500,0.9375,4,KLI1006-01-00
985319,O,20192,1,0,0,4,0,1,0.7500,0.8750,4,KLI1006-01-00


### mile df 학년 column dummy 변수화 시키기

In [329]:
temp = mile.copy()

In [330]:
temp['grade'].value_counts()

4    355826
2    342488
3    286770
1       237
Name: grade, dtype: int64

In [331]:
temp = pd.get_dummies(temp, prefix=['grade'], columns=['grade'])

In [332]:
temp

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,course_code,grade_1,grade_2,grade_3,grade_4
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,OMA1001-01-00,0,0,0,1
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,OMA1001-01-00,0,0,0,1
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,OMA1001-01-00,0,0,0,1
3,O,20161,36,0,0,5,0,1,0.7642,0.8421,OMA1001-01-00,0,0,0,1
4,O,20161,36,0,0,5,0,1,0.4337,0.4210,OMA1001-01-00,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985316,O,20192,1,0,0,5,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1
985317,O,20192,1,0,0,5,0,1,0.2500,0.9375,KLI1006-01-00,0,1,0,0
985318,O,20192,1,0,0,4,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1
985319,O,20192,1,0,0,4,0,1,0.7500,0.8750,KLI1006-01-00,0,0,0,1


In [333]:
mile = temp.copy()

## meta data

### meta df 복수전공 여부에 따라서 0, 1로 바꿔주기

In [ ]:
meta = meta.rename(columns={'grande_1':'grade_quota_1',
                           'grade_2':'grade_quota_2',
                           'grade_3':'grade_quota_3',
                           'grade_4':'grade_quota_4',})

In [ ]:
dict_ny = {'N':0, "Y":1}

In [ ]:
meta['second_major'] = meta['second_major'].map(dict_ny)

## Data 저장하기

In [411]:
mile.to_csv('mile(EDAed).csv', index=False)
meta.to_csv('meta(EDAed).csv', index=False)

# 4. Merge 시켜주기

In [2]:
mile = pd.read_csv('mile(EDAed).csv')
meta = pd.read_csv('meta(EDAed).csv')

In [3]:
mile.head(3)

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,course_code,grade_1,grade_2,grade_3,grade_4
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,OMA1001-01-00,0,0,0,1
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,OMA1001-01-00,0,0,0,1
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,OMA1001-01-00,0,0,0,1


In [5]:
meta.head(3)

,hyhg,course_code,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,20161,OMA1001-01-00,"김종철,홍기원",63,40,0,0,0,21,21,21,36,12.70
1,20161,OMA1002-01-00,우환식,63,98,0,0,0,21,21,21,36,20.10
2,20161,YCA1101-10-00,조재국,53,43,0,0,0,18,17,18,36,12.98


## meta df 중복값 없애고서 merge

In [428]:
temp = meta.copy()

In [429]:
temp

,hyhg,course_code,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,20161,OMA1001-01-00,"김종철,홍기원",63,40,0,0,0,21,21,21,36,12.70
1,20161,OMA1002-01-00,우환식,63,98,0,0,0,21,21,21,36,20.10
2,20161,YCA1101-10-00,조재국,53,43,0,0,0,18,17,18,36,12.98
3,20161,YCA1101-11-00,박정세,24,48,0,0,0,8,8,8,36,16.56
4,20161,YCA1101-12-00,민경식,50,14,50,0,0,0,0,0,36,7.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26581,20192,KLI1002-01-00,김성숙,80,45,0,0,0,0,0,0,36,5.47
26582,20192,KLI1003-01-00,김성숙,50,32,0,0,0,0,0,0,36,3.34
26583,20192,KLI1004-01-00,김성숙,40,24,0,0,0,0,0,0,36,4.08
26584,20192,KLI1005-01-00,김성숙,15,9,0,0,0,0,0,0,36,1.67


In [431]:
temp = temp.drop_duplicates(subset=list(temp.columns))

In [432]:
temp

,hyhg,course_code,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,20161,OMA1001-01-00,"김종철,홍기원",63,40,0,0,0,21,21,21,36,12.70
1,20161,OMA1002-01-00,우환식,63,98,0,0,0,21,21,21,36,20.10
2,20161,YCA1101-10-00,조재국,53,43,0,0,0,18,17,18,36,12.98
3,20161,YCA1101-11-00,박정세,24,48,0,0,0,8,8,8,36,16.56
4,20161,YCA1101-12-00,민경식,50,14,50,0,0,0,0,0,36,7.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26581,20192,KLI1002-01-00,김성숙,80,45,0,0,0,0,0,0,36,5.47
26582,20192,KLI1003-01-00,김성숙,50,32,0,0,0,0,0,0,36,3.34
26583,20192,KLI1004-01-00,김성숙,40,24,0,0,0,0,0,0,36,4.08
26584,20192,KLI1005-01-00,김성숙,15,9,0,0,0,0,0,0,36,1.67


## 학년별 정원 제한 없는 과목은 NaN값으로 바꿔주기

In [436]:
temp

,hyhg,course_code,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,20161,OMA1001-01-00,"김종철,홍기원",63,40,0,0,0,21,21,21,36,12.70
1,20161,OMA1002-01-00,우환식,63,98,0,0,0,21,21,21,36,20.10
2,20161,YCA1101-10-00,조재국,53,43,0,0,0,18,17,18,36,12.98
3,20161,YCA1101-11-00,박정세,24,48,0,0,0,8,8,8,36,16.56
4,20161,YCA1101-12-00,민경식,50,14,50,0,0,0,0,0,36,7.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
26581,20192,KLI1002-01-00,김성숙,80,45,0,0,0,0,0,0,36,5.47
26582,20192,KLI1003-01-00,김성숙,50,32,0,0,0,0,0,0,36,3.34
26583,20192,KLI1004-01-00,김성숙,40,24,0,0,0,0,0,0,36,4.08
26584,20192,KLI1005-01-00,김성숙,15,9,0,0,0,0,0,0,36,1.67


In [439]:
temp.loc[(temp['grade_quota_1'] ==0) &
         (temp['grade_quota_2'] ==0) &
         (temp['grade_quota_3'] ==0) &
         (temp['grade_quota_4'] ==0), ['grade_quota_1', 'grade_quota_2',
                                       'grade_quota_3', 'grade_quota_4']] = None

## merge 해주기

In [441]:
temp_df = pd.merge(mile, temp, how='left', on=['course_code','hyhg'])

In [442]:
temp_df.shape, mile.shape, temp.shape

((985592, 26), (985321, 15), (20747, 13))

In [443]:
temp_df.isnull().sum()

enrolled                      0
hyhg                          0
mileage                       0
major                         0
double                        0
enrolled_courses              0
graduation                    0
first_enroll                  0
credits_rate                  0
previous_credits_rate         0
course_code                   0
grade_1                       0
grade_2                       0
grade_3                       0
grade_4                       0
instructor                 1631
quota                        89
participants                 89
major_quota                  89
second_major                 89
grade_quota_1            403436
grade_quota_2            403436
grade_quota_3            403436
grade_quota_4            403436
max_mileage                  89
average                      89
dtype: int64

## meta data가 없는 row들 없애주기

In [448]:
temp_dropna = temp_df.copy()

In [450]:
temp_dropna.shape, temp_df.shape

((985592, 26), (985592, 26))

In [451]:
temp_dropna = temp_dropna.dropna(subset=['quota', 'participants'])

In [452]:
temp_dropna.isnull().sum()

enrolled                      0
hyhg                          0
mileage                       0
major                         0
double                        0
enrolled_courses              0
graduation                    0
first_enroll                  0
credits_rate                  0
previous_credits_rate         0
course_code                   0
grade_1                       0
grade_2                       0
grade_3                       0
grade_4                       0
instructor                 1542
quota                         0
participants                  0
major_quota                   0
second_major                  0
grade_quota_1            403347
grade_quota_2            403347
grade_quota_3            403347
grade_quota_4            403347
max_mileage                   0
average                       0
dtype: int64

In [456]:
temp_dropna

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,course_code,grade_1,grade_2,grade_3,grade_4,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
3,O,20161,36,0,0,5,0,1,0.7642,0.8421,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
4,O,20161,36,0,0,5,0,1,0.4337,0.4210,OMA1001-01-00,0,0,1,0,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985587,O,20192,1,0,0,5,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0
985588,O,20192,1,0,0,5,0,1,0.2500,0.9375,KLI1006-01-00,0,1,0,0,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0
985589,O,20192,1,0,0,4,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0
985590,O,20192,1,0,0,4,0,1,0.7500,0.8750,KLI1006-01-00,0,0,0,1,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0


## 데이터 저장하기

In [458]:
temp_dropna.to_csv('merged_1st.csv', index=False)

# 5. 데이터 확인

In [8]:
final_df = pd.read_csv('merged_1st.csv')

In [10]:
final_df

,enrolled,hyhg,mileage,major,double,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,course_code,grade_1,grade_2,grade_3,grade_4,instructor,quota,participants,major_quota,second_major,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4,max_mileage,average
0,O,20161,36,0,0,6,0,1,0.7925,0.9473,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
1,O,20161,36,0,0,6,0,1,0.7857,0.7142,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
2,O,20161,36,0,0,6,0,1,0.7428,0.8947,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
3,O,20161,36,0,0,5,0,1,0.7642,0.8421,OMA1001-01-00,0,0,0,1,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
4,O,20161,36,0,0,5,0,1,0.4337,0.4210,OMA1001-01-00,0,0,1,0,"김종철,홍기원",63.0,40.0,0.0,0.0,0.0,21.0,21.0,21.0,36.0,12.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985498,O,20192,1,0,0,5,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0
985499,O,20192,1,0,0,5,0,1,0.2500,0.9375,KLI1006-01-00,0,1,0,0,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0
985500,O,20192,1,0,0,4,0,1,0.7500,0.9375,KLI1006-01-00,0,0,0,1,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0
985501,O,20192,1,0,0,4,0,1,0.7500,0.8750,KLI1006-01-00,0,0,0,1,김성숙,15.0,5.0,0.0,0.0,NaN,NaN,NaN,NaN,36.0,1.0


In [11]:
final_df.columns

Index(['enrolled', 'hyhg', 'mileage', 'major', 'double', 'enrolled_courses',
       'graduation', 'first_enroll', 'credits_rate', 'previous_credits_rate',
       'course_code', 'grade_1', 'grade_2', 'grade_3', 'grade_4', 'instructor',
       'quota', 'participants', 'major_quota', 'second_major', 'grade_quota_1',
       'grade_quota_2', 'grade_quota_3', 'grade_quota_4', 'max_mileage',
       'average'],
      dtype='object')

In [12]:
final_df = final_df[['enrolled', 'hyhg', 'course_code','instructor', 'quota', 'participants',
                     'max_mileage', 'average', 'mileage','major', 'double',
                      'major_quota', 'second_major',
                     'enrolled_courses', 'graduation', 'first_enroll', 'credits_rate',
                     'previous_credits_rate',
                     'grade_1', 'grade_2', 'grade_3', 'grade_4', 
        'grade_quota_1','grade_quota_2', 'grade_quota_3', 'grade_quota_4'
       ]]

In [13]:
final_df

,enrolled,hyhg,course_code,instructor,quota,participants,max_mileage,average,mileage,major,double,major_quota,second_major,enrolled_courses,graduation,first_enroll,credits_rate,previous_credits_rate,grade_1,grade_2,grade_3,grade_4,grade_quota_1,grade_quota_2,grade_quota_3,grade_quota_4
0,O,20161,OMA1001-01-00,"김종철,홍기원",63.0,40.0,36.0,12.7,36,0,0,0.0,0.0,6,0,1,0.7925,0.9473,0,0,0,1,0.0,21.0,21.0,21.0
1,O,20161,OMA1001-01-00,"김종철,홍기원",63.0,40.0,36.0,12.7,36,0,0,0.0,0.0,6,0,1,0.7857,0.7142,0,0,0,1,0.0,21.0,21.0,21.0
2,O,20161,OMA1001-01-00,"김종철,홍기원",63.0,40.0,36.0,12.7,36,0,0,0.0,0.0,6,0,1,0.7428,0.8947,0,0,0,1,0.0,21.0,21.0,21.0
3,O,20161,OMA1001-01-00,"김종철,홍기원",63.0,40.0,36.0,12.7,36,0,0,0.0,0.0,5,0,1,0.7642,0.8421,0,0,0,1,0.0,21.0,21.0,21.0
4,O,20161,OMA1001-01-00,"김종철,홍기원",63.0,40.0,36.0,12.7,36,0,0,0.0,0.0,5,0,1,0.4337,0.4210,0,0,1,0,0.0,21.0,21.0,21.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
985498,O,20192,KLI1006-01-00,김성숙,15.0,5.0,36.0,1.0,1,0,0,0.0,0.0,5,0,1,0.7500,0.9375,0,0,0,1,NaN,NaN,NaN,NaN
985499,O,20192,KLI1006-01-00,김성숙,15.0,5.0,36.0,1.0,1,0,0,0.0,0.0,5,0,1,0.2500,0.9375,0,1,0,0,NaN,NaN,NaN,NaN
985500,O,20192,KLI1006-01-00,김성숙,15.0,5.0,36.0,1.0,1,0,0,0.0,0.0,4,0,1,0.7500,0.9375,0,0,0,1,NaN,NaN,NaN,NaN
985501,O,20192,KLI1006-01-00,김성숙,15.0,5.0,36.0,1.0,1,0,0,0.0,0.0,4,0,1,0.7500,0.8750,0,0,0,1,NaN,NaN,NaN,NaN


In [480]:
final_df['enrolled'].value_counts()

O    744699
X    240804
Name: enrolled, dtype: int64

최종 저장

In [14]:
final_df.to_csv('dataframe(mile,meta).csv', index=False)